In [ ]:
import Pkg; Pkg.activate("../")
Pkg.instantiate()

We will work with the following family of functions
\begin{align}
T_{\alpha,\beta}(x)=T(x,\alpha,\beta)&=\beta - (1+\beta)|x|^{\alpha},
\end{align}
where $\alpha\geq 1$, $\beta\in (-1,1]$ and $x\in [-1,1]$.
We will study the random dynamical system the family with Gaussian additive noise with variance $\sigma$ i.e
\begin{align*}
    X_{n+1}=\tau(T_{\alpha,\beta}(X_n)+\Omega_{\sigma}(n))
\end{align*}
where $\Omega_{\sigma}(n)$ are random variables i.i.d with Gaussian distribution and $\tau:\mathbb{R}\rightarrow [-1,1]$ boundary condition map $\tau(x)=\lfloor x \rfloor-1$, is the representative in $[0.2)$ of $x$ with repect to $x \sim y$ if $x-y=2k$, $k\in \mathbb{Z}$.
We want computing rigorously the Lyapunov exponent for these random dynamical systems
\begin{align*}
	\lambda(\alpha,\beta,\sigma)=\int _{-1}^{1}\ln |T'_{\alpha,\beta}|f_{\sigma}dm,
\end{align*}
where $T'_{\alpha,\beta}(x)=-\alpha (1+\beta)|x|^{\alpha-2}x$.

In [ ]:
using RigorousInvariantMeasures, IntervalArithmetic, BallArithmetic
α = interval(3.5)
β = interval(1)
K = 128
FFTNx = 1024

# this builds a fourier basis truncated at frequences [-K, K]
B = FourierAdjoint(K, FFTNx)                          

# this dynamics is defined on [-1, 1]
function T_alpha_beta_zeroone(α, β, x)                
    return β - (1 + β) * abs(x)^α
end

# coordinate change from [-1, 1] to [0, 1]
function τ_1(x)       
    return (x + 1) / 2
end

# coordinate change from [0, 1] to [-1, 1] 

function τ_2(x)     
    return 2 * x - 1
end

T_zeroone(x) = T_alpha_beta_zeroone(α, β, x)   # closure, fixing α and β

# dynamics on [0, 1]
T(x) = τ_1(T_zeroone(τ_2(x)))             

We will use the following approximation to find the value of the lyaponov exponent $λ(\alpha,\beta,σ)$.
Let $\alpha$ and $\beta$ be fixed, then
\begin{align*}
	|\lambda(\alpha,\beta,\sigma)-\lambda(\alpha,\beta,\sigma,k,s)|\leq \Upsilon \|f_{\sigma}-f_{\sigma,k,s}\|_{L^2}.
\end{align*}
where $f_{\sigma,k,s}$ is a symmetric trigonometric polynomial that approximates the fixed point of $P_{\sigma, K}$ and $\Upsilon = \sqrt{2}((\ln((1+\beta)\alpha)-(\alpha-1))^2+(\alpha-1)^2)^{\frac{1}{2}}$.

In [ ]:
Υ = sqrt(interval(2)) * ((log2((β + 1) * α) - (α - 1))^2 + (α - 1)^2)^(1 / 2)

To calculate $\lambda(\alpha,\beta,\sigma,k,s)$ we need $f_{\sigma,k,s}$ and therefore $f_{\sigma,k}$.

In [ ]:
Iπ = interval(π)

In [ ]:
using LinearAlgebra
# D é a matri que vai multiplicar a matriz do operador de tranferencia P para obter a matriz do operador de tranferecia com ruido (obs que D ja está trucanda)

σ = interval(0.1)

D = Diagonal([[exp((-σ^2 * π^2 * interval(k)^2) / 2) for k in 0:K]; [exp((-σ^2 * π^2 * interval(k)^2) / 2) for k in -K:-1]])
Dc, Dr = IntervalArithmetic.mid.(D), IntervalArithmetic.radius.(D)
bD = BallMatrix(Dc, Dr)

$P_{K}$ is the Galerkin truncation of the Perron-Frobenius operator (without noise)

In [ ]:
PK = assemble(B, T)        #aqui esta calculando el operador de transferencia finito de T, note que usa a matriz B que é a que converte o operador numa matriz na base de fourier                   #matriz de tranferencia com ruido y es finito, está en la base de Fourier

In [ ]:
# we convert this to a BallMatrix


center = IntervalArithmetic.mid.(real.(PK)) + im * IntervalArithmetic.mid.(imag.(PK))
radius = sqrt.(IntervalArithmetic.radius.(real.(PK)) .^ 2 + IntervalArithmetic.radius.(imag.(PK)) .^ 2)

bPK = BallMatrix(center, radius)

We compute now the annealed Perron-Frobenius operator

In [ ]:
PσK = bD * bPK

To calculate an approximation of the fixed point of the complex matrix of intervals $P\sigma K$, we take the matrix of the centers $A$
$$(A)_{ij}=mid(real(P_{ij}))+mid(im(P_{ij}))$$
where $mid$ takes the mid point of the interval. Now we use a numerical algorithm to compute an approximation of the fixed point of $A$.
Later, we are going to compute the residual of this fixed point.

In [ ]:
using LinearAlgebra

A = PσK.c 
F = eigen(A)                         

We plot the eigenvalues of the matrix $A$.

In [ ]:
using Plots
scatter(F.values, label = "Eigenvalues of A")
plot!([cos(x) for x in 0:0.01:2*pi], [sin(x) for x in 0:0.01:2*pi], label = "Unit circle")

The matrix F.vectors contains numerical approximations, not verified of the eigenvalues of $A$

In [ ]:
F.vectors               

In [ ]:
fσK = F.vectors[:, 257]   # this is going to be an approximation of our fixed point

$f_{\sigma,k}$ is fixed point of $A$

In [ ]:
fσK /= fσK[1] # we normalize it

In the computer calculations, the symmetry of the point was lost. Now let's symmetrize the fixed point

In [ ]:
fσKs = zeros(257) + zeros(257) * im
for i in 1:129
    fσKs[i] = fσK[i]
end

for i in 1:128
    fσKs[258-i] = conj(fσK[i+1])
end
fσKs                                


Note that
\begin{align*}
f_{\sigma,k,s}=(\mathcal{F}(f_{\sigma,k,s})[0],\mathcal{F}(f_{\sigma,k,s})[1],\ldots,\mathcal{F}(f_{\sigma,k,s})[K],\mathcal{F}(f_{\sigma,k,s})[-K],\ldots,\mathcal{F}(f_{\sigma,k,s})[-1])
\end{align*}


In [ ]:
bfσKs = BallVector(fσKs)

We compute now the residuals of the fixed point with respect to all the matrices in the BallMatrix

In [ ]:
residual = PσK * bfσKs - bfσKs

In [ ]:
ϵ = norm(residual.c, 2) + norm(residual.r, 2)

To compute the value of the Lyapunov exponent we need to find an enclosure of the Fourier coefficients of 
$$
\ln|T'_{\alpha,\beta}|
$$

I already have bounded $|\lambda(\alpha,\beta,\sigma)-\lambda(\alpha,\beta,\sigma,k,s)|$, it remains to find $\lambda(\alpha,\beta,\sigma,k,s)$ but 
\begin{align*}
\lambda (\alpha,\beta,\sigma,k,s)&=\int_{-1}^{1}\ln|T'_{\alpha,\beta}|f_{\sigma,k,s}dm\\
&=\langle \ln|T'_{\alpha,\beta}|,f_{\sigma,k,s} \rangle\\
&=\displaystyle \sum_{j=-k}^{k}\mathcal{F}(\ln|T'_{\alpha,\beta}|)[j]\mathcal{F}(f_{\sigma,k,s})[j].
\end{align*}
$\mathcal{F}(f_{\sigma,k})[j]$ we have already calculated it, let's calculate $\mathcal{F}(\ln|T'_{\alpha,\beta}|)[j]$.

We now that

\begin{align*}
\mathcal{F}(ln|T'_{\alpha,\beta}|)[0]&=\ln (2\alpha \beta)-(\alpha-1)\\
\mathcal{F}(ln|T'_{\alpha,\beta}|)[j]&=-\frac{(\alpha-1)}{j\pi}\int_{0}^{1}\frac{1}{x}sen(j\pi x)dx\\
&=-\frac{(\alpha-1)}{j\pi}\int _{0}^{j\pi}\frac{sen(x)}{x}dx.
\end{align*}

We want to compute
$$
\frac{1}{j\pi}\int_0^{j\pi} \frac{sin(t)}{t}dt
$$
for $j\geq 1$ by using Taylor Models integration for $[\pi, j\pi]$.

To make the computation more efficient, we will 
compute 
$$
I_i = \int_{i\pi}^{(i+1)\pi}\frac{sin(t)}{t}dt
$$

In [ ]:
f(t) = sin(t) / t

The  power series for $f$ at $x=0$ is
$$
f(x) = \sum_{i=0}^{+\infty}(-1)^i\frac{x^{2i}}{(2i+1)!}
$$
Then, the power series for the primitive is 
$$
F(x) = \sum_{i=0}^{+\infty}(-1)^i\frac{x^{2i+1}}{(2i+1)!(2i+1)}.
$$
We are going to use this Taylor series to approximate the integral only in the interval $[0, \pi]$.

We will bound the remainder of the series by using the alternating series remainder.

In [ ]:
function coeff_eval_at_0(i, x)
    return x^(2i + 1) / (factorial(big(2i + 1)) * (2i + 1))
end

In [ ]:
using IntervalArithmetic                                #No usa aqui Taylor Model porque no existe su serie de taylor pues la serie de taylor lo hace sacando la derivada y evaluando a en nuestro caro a=0, pero nosotros tenemos sen(t)/t el cual al derivar siempre tendra dividido por t^n y no se va poder remprazar para t=0
N = 1000

Pi = @biginterval π                                     # @interval(1)=[1,1], @interval(0.1)=[0.1,0.100001], @interval(x)= um intervalo fechado pequeno que contega x, @interval(1,2)=[1,2],@biginterval(1,2)=[1,2]_256 creo que va aceptar por ejemplo un decimal hasta la cifra 256
v = [(-1)^i * coeff_eval_at_0(i, Pi) for i in 0:N]        # como ja integro agora tem que valiar em \pi e 0, em verdad seria em jπ, mas ta tomando no intervalo [0,π] de momento
# i.e esta achando (-1)^{i}a_K, e os vota emcada coordenada de v, note que cada coordenadas é um intervalo pois Pi
error = Interval(-1, 1) * abs(coeff_eval_at_0(N + 1, Pi))   # error=[-a_{K+1},a_{K+1}], esto se tiene de Alternating series estimation theorem
@info error

integral_with_error = sum(v) + error                      # aqui esta sumando os primeros (-1)a_K + error e vai me sair um inervalo
integral_with_error, diam(integral_with_error)          # o diametri do intervalo que tem que ser muito pequeno

Therefore, we have computed $I_0$. We will now use 
Taylormodels to compute the value of the integrals
$$
\int_{i\pi}^{(i+1)\pi}\frac{sin(t)}{t}dt
$$
for $i>0$.

In [ ]:
#import Pkg; Pkg.add("TaylorModels")
using TaylorModels

In [ ]:
function integrate_i_i_plus_1(f, i; degree=40)            #aqui puede usar porque evalua las derivadas em m que es diferente de 0
    I = (@interval i * π (i + 1) * π)                             #esto es el intervalo [i*π,(i+1)*π], i>0
    m = Interval(IntervalArithmetic.mid(I))                                    #mid(I) ponto medio do intervalo I, Interval(mid(I))=un intervalo pequenho que contenga mid(I)
    tm = TaylorModel1(degree, m, I)                         #ese m es el valor que será evaluada en la serie de Taylos i.e f(m)+f'(m)t/1!+...+f^{(400)}(m)t^{400}/400!+erro
    prim = TaylorSeries.integrate(f(tm))                    #esto es la primitiva de f, al poner f(tm), recien esta diciendo quien va ser la función f para la cual voy hacer el polinomi de taylor i.e. para obtener la sumatoria que esta en la linea de arriba
    return prim(I.hi - m) - prim(I.lo - m)                        #I.hi, I.lo parte superior e inferior do intervalo I respectivamente, como m é um intervalo, vai ficar o intevalo prim([I.hi-m1,I.hi-m2])-prim([I.lo-m1,I.lo-m2]) where denotamos m=[m1,m2]
end

The vector `val` is containing at index $i$
the value of the integral
$$
\int_{i\pi}^{(i+1)\pi} \frac{\sin(t)}{t}dt
$$

In [ ]:
val = [integral_with_error; [integrate_i_i_plus_1(f, i) for i in 1:K-1]] #aqui junta I_0+el resultado de la integrales [π,2π],[2π,3π],...,[2000π,2001π], en un vector

In [ ]:
maximum(diam.(val))

The cumulative sum vector below contains at the $i$-th index the value of the integral
$$
\int_0^{i\pi}\frac{\sin(t)}{t}dt
$$

In [ ]:
cum_sum = cumsum(val)               #cumsum en cada coornenada suma lo anterior y lo pone ahí ejemplo v=(v_1,v_2,v_3,v_4)=(v_1,v_1+v_2,v_1+v_2+v_3,v_1+v_2+v_3+v_4)

The vector `coeff` contains at the index $i$ the value of
$$
\frac{1}{i\pi}\int_0^{i\pi} \frac{\sin(t)}{t}dt.
$$

In [ ]:
coeff = cum_sum ./ ([i * @interval π for i in 1:K])         #aqui solo le esta dividiendo por iπ a cada integral

I want
\begin{align*}
\mathcal{F}(ln|T'_{\alpha,\beta}|)[0]&=\ln ((1+\beta)\alpha)-(\alpha-1)\\
\mathcal{F}(ln|T'_{\alpha,\beta}|)[j]&=-\frac{(\alpha-1)}{j\pi}\int_{0}^{1}\frac{1}{x}sen(j\pi x)dx\\
&=-\frac{(\alpha-1)}{j\pi}\int _{0}^{j\pi}\frac{sen(x)}{x}dx.
\end{align*}
then

These are only the positive frequencies, so, we need to 
think about how to complete the vectore before taking the IFFT. 

In [ ]:
coeff_fft = [coeff; reverse(coeff[1:end])] #hallamos la mitad, ahora como es simetrico tamos escribiendo la otra mitad que falta ussando lo que ya tenemos

We need now to take into account the constants $\alpha$ and $\beta$

In [ ]:
ln = [log((1 + β) * α) - (α - 1); -(α - 1) * coeff_fft]  

Since these were the coefficients computed on the Fourier basis of $[-1,1]$ we need to convert them to the coefficients in the Fourier Basis of $[0,1]$,
i.e., we need to apply the coordinate change.

In [ ]:
lnn = zeros(Interval, 257) #2*K+1
lnn[1] = ln[1]
for i in 2:129
    lnn[i] = (-1)^(i + 1) * ln[i]
end
for i in 130:257
    lnn[i] = (-1)^(i) * ln[i]
end
IntervalArithmetic.mid.(lnn[1:10])

\begin{align*}
ln=(\mathcal{F}(\ln|T'_{\alpha,\beta}|)[0],\mathcal{F}(\ln|T'_{\alpha,\beta}|)[1],\ldots,\mathcal{F}(\ln|T'_{\alpha,\beta}|)[K],\mathcal{F}(\ln|T'_{\alpha,\beta}|)[-K],\ldots,\mathcal{F}(\ln|T'_{\alpha,\beta}|)[-1])
\end{align*}

Finally
\begin{align*}
\lambda (\alpha,\beta,\sigma,k)=\displaystyle \sum_{j=-k}^{k}\mathcal{F}(\ln|T'_{\alpha,\beta}|)[j]\mathcal{F}(f_{\sigma,k})[j].
\end{align*}


In [ ]:
λ_k = 0 + im * 0
for i in 1:2*K+1
    λ_k = lnn[i] * fσKs[i] + λ_k
end
λ_k  

Now to limit $\|f_{\sigma}-f_{\sigma,k,s}\|$ we use the following result

Let $f_{\sigma}$ be the unique fixed point of $L_{\sigma}$ and $f_{\sigma,k}$ be the unique fixed point of $L_{\sigma,k}$ and $f_{\sigma,k,s}$ symmetrization of $f_{k,s}$. Suppose that there exists $n \in \mathbb{N}$ and $\eta>0$ such that 
\begin{align*}
\|L_{\sigma,k }^{n}|_{V}\|_{L^{2}\rightarrow L^{2}}\leq \eta <1,	
\end{align*}
where $V$ be the zero average subspace of $L^{2}$, then if $1\leq C_i $ are such that $\|L^i_{\sigma,k}|_V\|_{L^{2}\rightarrow L^{2}}\leq C_i$ for $1\leq i \leq n-1$, we have
\begin{align*}
\|f_{\sigma}-f_{\sigma,k,s}\|_{L^{2}}\leq \frac{1}{1-\eta}\displaystyle \sum _{i=0}^{n-1}C_i((1+\Gamma_{\sigma,k}+\|\rho_{\sigma}\|_{L^{2}})\Gamma_{\sigma,k}\|f_{\sigma}\|_{L^1}+\epsilon).
\end{align*}

Remark that $\epsilon$ is the residual with respect to the approximation of the fixed point.

In [ ]:
σ = interval(σ)
Γ = ((1 / (sqrt(σ^2 * 2 * interval(π))))exp((-σ^2 * K^2 * interval(π)^2) / 2))   #para σ=0,1 e K=128, Γ=3.03*10^(-702)

We apply now the Theorem, by computing bounds for the norm of the operator restricted to the space of average $0$ measures.

In [ ]:
A = PσK[2:end, 2:end]

Aiter = A

C = zeros(10)
η = 0
n_0 = 0
for n in 1:10
    θ = BallArithmetic.svd_bound_L2_opnorm(Aiter)
    if θ > 1
        C[n] = θ
    else
        global η = θ
        global n_1 = n                                    #usso n_1 porque mas adelante usso n_0
        break                                           #para que pare y no siga corriendo n, sino el η me va salir más chico
    end
    Aiter *= A 
end
η, C

In [ ]:
ρ1 = 1 / (sqrt(σ^2 * 2 * interval(π)))
ρn = sqrt(ρ1)


In [ ]:
#ya tenho ϵ, os C_i, η,h, n_1 é dizer ate onde vai asumatoria, |u|=1, e usei PK o operador de tranferencia finito, pero creo que tiene que ser el operador de transferencia infinito, só me falta K
R_1 = 0                                             #ja tenho todos os valos que preciso ahora só resta fazer a sumatoria
for i in 1:n_1-1
    global R_1 = R_1 + C[i] * ((1 + Γ + ρn)Γ + ϵ)
end
R_1 = (1 / (1 - η)) * R_1

In [ ]:
#el intervalo donde estaria λ seria
@biginterval(real.(λ_k) - Υ * R_1, real.(λ_k) + Υ * R_1) + im * @biginterval(imag.(λ_k) - Υ * R_1, imag.(λ_k) + Υ * R_1)

We can observe that the integral must be a real number, so, we can restrict the enclosure to the real line

In [ ]:
@biginterval(real.(λ_k) - Υ * R_1, real.(λ_k) + Υ * R_1)